In [42]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [43]:
input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Url'].tolist()

总数量：2484


['https://portal.thepartshq.com/marmon-ride-control-a2041-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2053-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2091-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2092-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2120-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2122-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2123-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2142-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2167-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2220-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2221-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2226-engine-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2240-engine

In [44]:
crawler_status = 'error'

print(crawler_status)

error


In [45]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [46]:
a = 0

resp = requests.get(input_.loc[a, 'Url'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!doctype html>\n<html lang="en">\n    <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# product: http://ogp.me/ns/product#">\n        <script>\n    var LOCALE = \'en\\u002DUS\';\n    var BASE_URL = \'https\\u003A\\u002F\\u002Fportal.thepartshq.com\\u002F\';\n    var require = {\n        \'baseUrl\': \'https\\u003A\\u002F\\u002Fportal.thepartshq.com\\u002Fstatic\\u002Fversion1710219206\\u002Ffrontend\\u002FMarmon\\u002Funiversalparts\\u002Fen_US\'\n    };</script>        <meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"1120047568",accountID:"3406985",trustKey:"1322840",xpid:"VwIHV19bDRABVFVaDgEOVVQE",licenseKey:"NRJS-92b9060cf9045d99b7e",applicationID:"1048909305"};;/*! For license information please see nr-loader-spa-1.255.0.min.js.LICENSE.txt */\n(()=>{var e,t,r={234:(e,t,r)=>{"use 

In [47]:
from bs4 import BeautifulSoup
from lxml import etree

In [48]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./oe.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# product: http://ogp.me/ns/product#">
  <script>
   var LOCALE = 'en\u002DUS';
    var BASE_URL = 'https\u003A\u002F\u002Fportal.thepartshq.com\u002F';
    var require = {
        'baseUrl': 'https\u003A\u002F\u002Fportal.thepartshq.com\u002Fstatic\u002Fversion1710219206\u002Ffrontend\u002FMarmon\u002Funiversalparts\u002Fen_US'
    };
  </script>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"1120047568",accountID:"3406985",trustKey:"1322840",xpid:"VwIHV19bDRABVFVaDgEOVVQE",licenseKey:"NRJS-92b9060cf9045d99b7e",applicationID:"1048909305"};;/*! For license information please see nr-loader-spa-1.255.0.min.js.LICENSE.txt */
(()=>{var e,t,r={234:(e,t,r)=>{"use strict";r.d(t,{P_:()=>m,Mt:()=>b,C5:(

In [25]:
list_tr = []
list_div = html.xpath('//div[@class="product info detailed"]/div/div')
for i in range(len(list_div)):
    if len(list_div[i].xpath('./a/text()')) != 0 and list_div[i].xpath('./a/text()')[0].strip() == 'Cross Reference':
        list_tr = list_div[i+1].xpath('./table/tbody/tr')
        
list_tr

[<Element tr at 0x123a8bc40>,
 <Element tr at 0x123321600>,
 <Element tr at 0x123a205c0>]

In [26]:
df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number']}])

for tr in list_tr:
    df_temp.loc[0, 'OE_' + tr.xpath('./td[1]/text()')[0].strip()] = tr.xpath('./td[2]/text()')[0].strip()
    
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [27]:
crawler_status = 'ok'

print(crawler_status)

ok


In [28]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                             'Url': input_.loc[a, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [30]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_oe.xlsx', index=False)

output_correct

,Part_Number,OE_ANCHOR DOAN,OE_AUTOZONE-MEXICO,OE_GENERAL MOTORS
0,A2041,2041,A2041,2356569


In [31]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_oe_error.xlsx', index=False)

output_error

""
